# TP2: Machine learning

In [35]:
import pandas as pd
import time
%matplotlib inline
from datetime import datetime
import scipy.spatial
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

In [2]:
data_training = pd.read_csv('data/data_update-2017-10.csv')

In [3]:
data_training = data_training[data_training['state_name'].str.contains('G.B.A') | data_training['state_name'].str.contains('Capital Federal')]
##El tp dice solo gba y caba

In [4]:
def recuperar_cant_amb(x):
    "Recupera el dato amb en x (puede ser la descripcion o el titulo)"
    try:
        x = x.lower()
        if x.rfind("ambientes") != -1 or x.rfind("amb") != -1:
            pos = x.rfind("amb")
            try:
                return (float(x[pos -2])) # intento convertir lo que hay antes de la palabra
                                              #si funciona es el numero de rooms
            except ValueError:
                if "dos amb" in x:
                    return 2.0
                elif "tres amb" in x:
                    return 3.0
                elif "cuatro amb" in x:
                    return 4.0
                elif "cinco amb" in x:
                    return 5.0
                elif "seis amb" in x:
                    return 6.0
                return 0
    except AttributeError:
        return None

In [5]:
def buscar_servicios_en_descripcion(descripcion):
    """Devuelve un diccionario con 1 o 0 segun cuente con el servicio o no.
    Los servicios que consideraremos son cochera,pileta, vigilancia y gimnasio"""
    dic = {"cochera":0,"pileta":0,"vigilancia":0,"gimnasio":0,"patio":0,"terraza":0,"parrilla":0,"lavadero":0}
    descripcion = str(descripcion)
    descripcion = str.lower(descripcion)
    if "cochera" in descripcion or "garage" in descripcion:
        dic["cochera"] = 1
    elif "pileta" in descripcion or "piscina" in descripcion:
        dic["pileta"] = 1
    elif "vigilancia" in descripcion:
        dic["vigilancia"] = 1
    elif "gimnasio" in descripcion:
        dic["gimnasio"] = 1
    elif "patio" in descripcion:
        dic["patio"] = 1
    elif "terraza" in descripcion:
        dic["terraza"] = 1
    elif "parrilla" in descripcion:
        dic["parrilla"] = 1
    elif "lavadero" in descripcion:
        dic["lavadero"] = 1
    return dic

In [6]:
def contar_ambientes(x):
    "Define la cantidad de ambientes en la descripcion"
    contador = 0
    try:
        x = x.lower()
        if x.rfind("dormitorios") != -1: 
            pos = x.rfind("dormitorios")
        elif x.rfind("habitaciones") != -1:
            pos = x.rfind("habitaciones")
            try:
                contador += (float(x[pos -2])) # intento convertir lo que hay antes de la palabra
                                              #si funciona es el numero de rooms
            except ValueError:
                if ("dos dormitorios" in x) or ("dos habitaciones" in x):
                    contador += 2.0
                elif ("tres dormitorios" in x) or ("tres habitaciones" in x):
                    contador += 3.0
                elif ("cuatro dormitorios" in x) or ("cuatro habitaciones" in x):
                    contador += 4.0
                elif ("cinco dormitorios" in x) or ("cinco habitaciones" in x):
                    contador += 5.0
                elif ("seis dormitorios" in x) or ("seis habitaciones" in x):
                    contador += 6.0
        if ("living" or "comedor") in x:
            contador += 1.0
    except AttributeError:
        return None
    return contador

In [7]:
def place_name_from_parent(x): ## Segun vi el 3 item del place es generalmente el lugar
    list = x.split('|')
    if(len(list) < 5):
        return list[2]
    else:
        return list[3]

In [8]:
#Hay registros con precio en usd 0, pero en su descripcion se encuentra el precio asi que lo rescatamos
def rescatar_precio_descrip(descripcion):
    try:
        descripcion = str.lower(descripcion)
    except TypeError:
        #No tiene descripcion
        return 0
    try:
        if descripcion.rfind("usd") != -1 and descripcion.rfind("null") != -1:
            pos_usd = descripcion.rfind("usd")
            pos_null = descripcion.rfind("null")
            precio = (descripcion[pos_usd + 4 : pos_null -1]).lstrip().rstrip()
            if precio.isnumeric():
                return float(precio)
            elif descripcion.rfind('codigo: 2007-60') != -1:
                #Es un registro que no puedo recuperar genericamente
                return 35000
            else:
                #No es del formato USD precio null
                return 0
    except AttributeError:
        return 0

In [9]:
#Inicializo columnas
data_training["servicios"] = np.nan
data_training["cochera"] = int(0)
data_training["pileta"] = int(0)
data_training["vigilancia"] = int(0)
data_training["gimnasio"] = int(0)
data_training["patio"] = int(0)
data_training["terraza"] = int(0)
data_training["parrilla"] = int(0)
data_training["lavadero"] = int(0)

In [10]:
#Busco en las descripciones
data_training.loc[data_training["description"].notnull(),"servicios"] = data_training[data_training["description"].notnull()].description.apply(buscar_servicios_en_descripcion)

In [11]:
#Modifico los valores encontrados en la tabla
data_training.loc[:,"cochera"] = data_training[data_training["servicios"].notnull()].apply(lambda x: x.servicios["cochera"],axis = 1)
data_training.loc[:,"pileta"] = data_training[data_training["servicios"].notnull()].apply(lambda x: x.servicios["pileta"],axis=1)
data_training.loc[:,"vigilancia"] = data_training[data_training["servicios"].notnull()].apply(lambda x: x.servicios["vigilancia"],axis=1)
data_training.loc[:,"gimnasio"] = data_training[data_training["servicios"].notnull()].apply(lambda x: x.servicios["gimnasio"],axis=1)
data_training.loc[:,"patio"] = data_training[data_training["servicios"].notnull()].apply(lambda x: x.servicios["patio"],axis = 1)
data_training.loc[:,"terraza"] = data_training[data_training["servicios"].notnull()].apply(lambda x: x.servicios["terraza"],axis=1)
data_training.loc[:,"parrilla"] = data_training[data_training["servicios"].notnull()].apply(lambda x: x.servicios["parrilla"],axis=1)
data_training.loc[:,"lavadero"] = data_training[data_training["servicios"].notnull()].apply(lambda x: x.servicios["lavadero"],axis=1)

In [12]:
#Hay 12 campos que no se porque no se rellenan con 0
data_training.cochera.fillna(0,inplace=True)
data_training.pileta.fillna(0,inplace=True)
data_training.vigilancia.fillna(0,inplace=True)
data_training.gimnasio.fillna(0,inplace=True)
data_training.patio.fillna(0,inplace=True)
data_training.terraza.fillna(0,inplace=True)
data_training.parrilla.fillna(0,inplace=True)
data_training.lavadero.fillna(0,inplace=True)

In [13]:
#Elimino la columna servicios que ya no me sirve
data_training.drop("servicios",1,inplace=True)
data_training.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 317039 entries, 0 to 317038
Data columns (total 31 columns):
id                            317039 non-null object
created_on                    317039 non-null object
property_type                 317039 non-null object
place_name                    316958 non-null object
place_with_parent_names       317039 non-null object
state_name                    317039 non-null object
geonames_id                   251947 non-null float64
lat-lon                       224872 non-null object
lat                           224872 non-null float64
lon                           224872 non-null float64
price                         295111 non-null float64
currency                      287249 non-null object
price_aprox_local_currency    295111 non-null float64
price_aprox_usd               295111 non-null float64
surface_total_in_m2           234338 non-null float64
surface_covered_in_m2         279930 non-null float64
price_usd_per_m2              201

In [14]:
data_training.loc[data_training['price_aprox_usd'] == 0,'price_aprox_usd'] = data_training.loc[data_training['price_aprox_usd'] == 0].description.apply(rescatar_precio_descrip)

In [15]:
data_training.dropna(axis=0,subset=['price_aprox_usd'],inplace=True) ## drop de los datos sin precio
data_training = data_training.loc[data_training['price_aprox_usd'] > 0,:]

#Recuperamos el dato rooms en el titulo y la descripcion
data_training.loc[data_training.rooms.isnull(),'rooms'] = data_training.loc[data_training.rooms.isnull()].description.apply(recuperar_cant_amb)
data_training.loc[data_training.rooms.isnull(),'rooms'] = data_training.loc[data_training.rooms.isnull()].title.apply(recuperar_cant_amb)
#Recuperamos el dato rooms en la descripcion contando ambientes (living y dormitorios)
data_training.loc[data_training.rooms.isnull(),'rooms'] = data_training.loc[data_training.rooms.isnull()].description.apply(contar_ambientes)
data_training.loc[data_training.rooms == 0,'rooms'] = data_training.loc[data_training.rooms == 0].description.apply(contar_ambientes)


data_training.loc[:,'place_name'] = data_training.place_with_parent_names.apply(place_name_from_parent)
## esto quita muchos problemas de place name como nombres de country o palermo soho

In [16]:
data_training = data_training.dropna(axis = 0, subset = ['price_aprox_usd','surface_total_in_m2'],how ='any')
data_training['year'] = 0
data_training['month'] = 0
data_training.loc[:,'created_on'] = data_training.created_on.apply(pd.to_datetime)

In [17]:
data_training.loc[:,'year'] = data_training.created_on.apply(lambda x: x.year)
data_training.loc[:,'month'] = data_training.created_on.apply(lambda x: x.month)
data_training.drop('created_on',1,inplace=True)

In [18]:
#Hay 2 registro con rooms nan irrecuperables
data_training.drop(132299,inplace=True)
data_training.drop(95386,inplace=True)

In [19]:
#NO considerare lat lon porque reduce mucho los registros
data_training.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212220 entries, 0 to 317037
Data columns (total 32 columns):
id                            212220 non-null object
property_type                 212220 non-null object
place_name                    212220 non-null object
place_with_parent_names       212220 non-null object
state_name                    212220 non-null object
geonames_id                   167940 non-null float64
lat-lon                       151939 non-null object
lat                           151939 non-null float64
lon                           151939 non-null float64
price                         212220 non-null float64
currency                      212220 non-null object
price_aprox_local_currency    212220 non-null float64
price_aprox_usd               212220 non-null float64
surface_total_in_m2           212220 non-null float64
surface_covered_in_m2         202762 non-null float64
price_usd_per_m2              201207 non-null float64
price_per_m2                  19

In [20]:
#Recuperamos registros a los que no se les calculo el precio por m2 en dolares
data_training.loc[data_training['price_usd_per_m2'].isnull(),'price_usd_per_m2'] = data_training.surface_total_in_m2 / data_training.price_aprox_usd
data_training.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212220 entries, 0 to 317037
Data columns (total 32 columns):
id                            212220 non-null object
property_type                 212220 non-null object
place_name                    212220 non-null object
place_with_parent_names       212220 non-null object
state_name                    212220 non-null object
geonames_id                   167940 non-null float64
lat-lon                       151939 non-null object
lat                           151939 non-null float64
lon                           151939 non-null float64
price                         212220 non-null float64
currency                      212220 non-null object
price_aprox_local_currency    212220 non-null float64
price_aprox_usd               212220 non-null float64
surface_total_in_m2           212220 non-null float64
surface_covered_in_m2         202762 non-null float64
price_usd_per_m2              212220 non-null float64
price_per_m2                  19

In [21]:
#Vamos a codificar cada columna que sea de strings
lb_make1 = LabelEncoder()
data_training["property_type_code"] = lb_make1.fit_transform(data_training["property_type"])

lb_make2 = LabelEncoder()
data_training["place_name_code"] = lb_make2.fit_transform(data_training["place_name"])

lb_make3 = LabelEncoder()
data_training["place_with_parent_names_code"] = lb_make3.fit_transform(data_training["place_with_parent_names"])

lb_make4 = LabelEncoder()
data_training["state_name_code"] = lb_make4.fit_transform(data_training["state_name"])

In [22]:
data_training2 = data_training[['lat', 'lon', 'price_aprox_usd', 'surface_total_in_m2',
       'price_usd_per_m2', 'rooms', 'year', 'month', 'property_type_code',
       'place_name_code', 'place_with_parent_names_code', 'state_name_code']]

data_training2 = data_training2.dropna(axis = 0, subset = ['lat','lon'],how ='any')

In [23]:
data_training3 = data_training[['price_aprox_usd','lat','lon','surface_total_in_m2',
       'rooms', 'year', 'month', 'property_type_code','place_name_code', 
        'place_with_parent_names_code', 'state_name_code','cochera', 'pileta', 'vigilancia',
        'gimnasio', 'patio', 'terraza','parrilla', 'lavadero']]
data_training3 = data_training3.dropna(axis = 0, subset = ['lat','lon'],how ='any')

In [24]:
data_training3.drop(266756,inplace = True)

In [25]:
data_training3.corr().price_aprox_usd.sort_values(ascending = False)

price_aprox_usd                 1.000000
surface_total_in_m2             0.248762
property_type_code              0.202568
rooms                           0.132426
cochera                         0.103674
pileta                          0.081541
place_name_code                 0.063113
year                            0.044145
lat                             0.020579
month                           0.017958
lon                             0.011514
vigilancia                      0.002430
gimnasio                        0.000185
parrilla                       -0.018558
place_with_parent_names_code   -0.035253
terraza                        -0.035311
state_name_code                -0.042031
lavadero                       -0.058642
patio                          -0.066454
Name: price_aprox_usd, dtype: float64

In [26]:
columnas_corr = ['surface_total_in_m2','rooms','year','property_type_code','place_name_code','cochera','pileta']

['surface_total_in_m2',
 'rooms',
 'year',
 'property_type_code',
 'place_name_code',
 'cochera',
 'pileta',
 'price_aprox_usd']

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import HuberRegressor 
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LassoLars
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import BayesianRidge

dataframe = data_training3.drop(['price_aprox_usd'],axis=1)

columnas = columnas_corr
columnas_precio = columnas + ['price_aprox_usd']

set_entrenamiento = data_training3.loc[(data_training3.year >= 2016)]
set_entrenamiento = set_entrenamiento[columnas_precio]
set_pruebas = data_training3.loc[(data_training3.year >= 2016)]

array = set_entrenamiento.values
X = array[:,0:7]
Y = array[:,7]

# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
models.append(('Linear Regression', LinearRegression()))
models.append(('Ridge', Ridge()))
models.append(('RidgeCV', RidgeCV()))
models.append(('Lasso', Lasso()))
models.append(('Elastic Net', ElasticNet()))
models.append(('LassoLars', LassoLars()))
models.append(('OrthogonalMatchingPursuit', OrthogonalMatchingPursuit()))
models.append(('BayesianRidge', BayesianRidge()))
models.append(('HuberRegressor  ', HuberRegressor()))

# evaluate each model in turn
for name, model in models:
    model.fit(X,Y)
    score = model.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
    print("Precision = {:.2f} % , name = {}".format(score,name))


Precision = 17.15 % , name = Linear Regression
Precision = 17.15 % , name = Ridge
Precision = 17.15 % , name = RidgeCV
Precision = 17.15 % , name = Lasso
Precision = 15.04 % , name = Elastic Net
Precision = 17.15 % , name = LassoLars
Precision = 9.58 % , name = OrthogonalMatchingPursuit
Precision = 17.15 % , name = BayesianRidge
Precision = -4.41 % , name = HuberRegressor  


# Formateo el testing set

In [53]:
testingSet = pd.read_csv("data/properati_dataset_testing_noprice.csv")

In [54]:
testingSet = testingSet[testingSet['state_name'].str.contains('G.B.A') | testingSet['state_name'].str.contains('Capital Federal')]
##El tp dice solo gba y caba

In [29]:
testingSet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14165 entries, 0 to 14165
Data columns (total 17 columns):
id                         14165 non-null int64
created_on                 14165 non-null object
property_type              14165 non-null object
operation                  14165 non-null object
place_name                 14165 non-null object
place_with_parent_names    14165 non-null object
country_name               14165 non-null object
state_name                 14165 non-null object
lat-lon                    10487 non-null object
lat                        10487 non-null float64
lon                        10487 non-null float64
surface_total_in_m2        11852 non-null float64
surface_covered_in_m2      13005 non-null float64
floor                      1368 non-null float64
rooms                      7500 non-null float64
expenses                   2543 non-null object
description                14165 non-null object
dtypes: float64(6), int64(1), object(10)
memory usage: 1

In [55]:
#Inicializo columnas
testingSet["servicios"] = np.nan
testingSet["cochera"] = int(0)
testingSet["pileta"] = int(0)
testingSet["vigilancia"] = int(0)
testingSet["gimnasio"] = int(0)
testingSet["patio"] = int(0)
testingSet["terraza"] = int(0)
testingSet["parrilla"] = int(0)
testingSet["lavadero"] = int(0)

In [56]:
#Busco en las descripciones
testingSet.loc[testingSet["description"].notnull(),"servicios"] = testingSet[testingSet["description"].notnull()].description.apply(buscar_servicios_en_descripcion)

In [57]:
#Modifico los valores encontrados en la tabla
testingSet.loc[:,"cochera"] = testingSet[testingSet["servicios"].notnull()].apply(lambda x: x.servicios["cochera"],axis = 1)
testingSet.loc[:,"pileta"] = testingSet[testingSet["servicios"].notnull()].apply(lambda x: x.servicios["pileta"],axis=1)
testingSet.loc[:,"vigilancia"] = testingSet[testingSet["servicios"].notnull()].apply(lambda x: x.servicios["vigilancia"],axis=1)
testingSet.loc[:,"gimnasio"] = testingSet[testingSet["servicios"].notnull()].apply(lambda x: x.servicios["gimnasio"],axis=1)
testingSet.loc[:,"patio"] = testingSet[testingSet["servicios"].notnull()].apply(lambda x: x.servicios["patio"],axis = 1)
testingSet.loc[:,"terraza"] = testingSet[testingSet["servicios"].notnull()].apply(lambda x: x.servicios["terraza"],axis=1)
testingSet.loc[:,"parrilla"] = testingSet[testingSet["servicios"].notnull()].apply(lambda x: x.servicios["parrilla"],axis=1)
testingSet.loc[:,"lavadero"] = testingSet[testingSet["servicios"].notnull()].apply(lambda x: x.servicios["lavadero"],axis=1)

In [58]:
#Elimino la columna servicios que ya no me sirve
testingSet.drop("servicios",1,inplace=True)
testingSet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14165 entries, 0 to 14165
Data columns (total 25 columns):
id                         14165 non-null int64
created_on                 14165 non-null object
property_type              14165 non-null object
operation                  14165 non-null object
place_name                 14165 non-null object
place_with_parent_names    14165 non-null object
country_name               14165 non-null object
state_name                 14165 non-null object
lat-lon                    10487 non-null object
lat                        10487 non-null float64
lon                        10487 non-null float64
surface_total_in_m2        11852 non-null float64
surface_covered_in_m2      13005 non-null float64
floor                      1368 non-null float64
rooms                      7500 non-null float64
expenses                   2543 non-null object
description                14165 non-null object
cochera                    14165 non-null int64
pileta  

In [59]:
#Recuperamos el dato rooms en la descripcion
testingSet.loc[testingSet.rooms.isnull(),'rooms'] = testingSet.loc[testingSet.rooms.isnull()].description.apply(recuperar_cant_amb)

#Recuperamos el dato rooms en la descripcion contando ambientes (living y dormitorios)
testingSet.loc[testingSet.rooms.isnull(),'rooms'] = testingSet.loc[testingSet.rooms.isnull()].description.apply(contar_ambientes)
testingSet.loc[testingSet.rooms == 0,'rooms'] = testingSet.loc[testingSet.rooms == 0].description.apply(contar_ambientes)


testingSet.loc[:,'place_name'] = testingSet.place_with_parent_names.apply(place_name_from_parent)
## esto quita muchos problemas de place name como nombres de country o palermo soho

In [60]:
testingSet['year'] = 0
testingSet['month'] = 0
testingSet.loc[:,'created_on'] = testingSet.created_on.apply(pd.to_datetime)
testingSet.loc[:,'year'] = testingSet.created_on.apply(lambda x: x.year)
testingSet.loc[:,'month'] = testingSet.created_on.apply(lambda x: x.month)
testingSet.drop('created_on',1,inplace=True)

In [61]:
testingSet.loc[testingSet.property_type == 'casa','property_type'] = 'house'
testingSet.loc[testingSet.property_type == 'departamento','property_type'] = 'apartment'
testingSet.loc[testingSet.property_type == 'ph','property_type'] = 'PH'

In [62]:
#Vamos a codificar cada columna que sea de strings
testingSet["property_type_code"] = lb_make1.transform(testingSet["property_type"])

testingSet["place_name_code"] = lb_make2.transform(testingSet["place_name"])

testingSet["state_name_code"] = lb_make4.transform(testingSet["state_name"])

In [63]:
#Vamos a codificar cada columna que sea de strings
testingSet["property_type_code"] = lb_make1.transform(testingSet["property_type"])

testingSet["place_name_code"] = lb_make2.transform(testingSet["place_name"])

#testingSet["place_with_parent_names_code"] = lb_make3.transform(testingSet["place_with_parent_names"])

testingSet["state_name_code"] = lb_make4.transform(testingSet["state_name"])

178000.0

# Decision trees

In [39]:
from sklearn.tree import DecisionTreeRegressor

#X, y = data_training3, data_training3.price_aprox_usd  ###Da menos % usando todo el dataset
X,y = data_training3.loc[(data_training3.year >= 2016)], (data_training3.loc[(data_training3.year >= 2016)]).price_aprox_usd
X = X[columnas_corr]

regression_tree = DecisionTreeRegressor()
regression_tree.fit(X,y)
precission = regression_tree.score(X,y) * 100
mse = np.mean(cross_val_score(regression_tree, X, y,scoring='neg_mean_squared_error', cv=10,  n_jobs=1))
print("Method name = {}".format("DecisionTreeRegressor"))
print ('Mean squared error: {}'.format(abs(mse)))
print ('Precission: {:.2f} %'. format(abs(precission)))

Method name = DecisionTreeRegressor
Mean squared error: 1.01615773935e+11
Precission: 95.11 %


# RandomForest

In [45]:
from sklearn.ensemble import RandomForestRegressor

#X, y = data_training3, data_training3.price_aprox_usd  ###Da menos % usando todo el dataset
X,y = data_training3.loc[(data_training3.year >= 2016)], (data_training3.loc[(data_training3.year >= 2016)]).price_aprox_usd
columnas2 = ['lat', 'lon', 'surface_total_in_m2', 'rooms', 'year', 'month',
 'property_type_code', 'place_name_code', 'state_name_code', 'cochera', 'pileta', 'vigilancia', 'gimnasio',
 'patio', 'terraza', 'parrilla', 'lavadero']
#Aca funciona mejor usar todas las columnas a usar solo las mejores en corr
X = X[columnas2]

random_forest = RandomForestRegressor()
random_forest.fit(X,y)
precission = random_forest.score(X,y) * 100
mse = np.mean(cross_val_score(random_forest, X, y,scoring='neg_mean_squared_error', cv=15,  n_jobs=2))
print("Method name = {}".format("RandomForestRegressor"))
print ('Mean squared error: {}'.format(abs(mse)))
print ('Precission: {:.2f} %'. format(abs(precission)))

Method name = RandomForestRegressor
Mean squared error: 60261323777.6
Precission: 92.97 %


# ExtraTrees

In [47]:
from sklearn.ensemble import ExtraTreesRegressor

X,y = data_training3.loc[(data_training3.year >= 2016)], (data_training3.loc[(data_training3.year >= 2016)]).price_aprox_usd
X = X[columnas2]

extra_tree = ExtraTreesRegressor()
extra_tree.fit(X,y)


precission = extra_tree.score(X,y) * 100
mse = np.mean(cross_val_score(extra_tree, X, y,scoring='neg_mean_squared_error', cv=15,  n_jobs=2))
print("Method name = {}".format("ExtraTreesRegressor"))
print ('Mean squared error: {}'.format(abs(mse)))
print ('Precission: {} %'. format(abs(precission)))

Mean squared error: 66088860386.5
Precission: 99.5539596351 %


# GradientBoosting

In [82]:
from sklearn.ensemble import GradientBoostingRegressor

#X, y = data_training3, data_training3.price_aprox_usd  ###Da menos % usando todo el dataset
X,y = data_training3.loc[(data_training3.year >= 2016)], (data_training3.loc[(data_training3.year >= 2016)]).price_aprox_usd
X = X[columnas2]

gradient_boosting = GradientBoostingRegressor()
gradient_boosting.fit(X,y)

precission = gradient_boosting.score(X,y) * 100
mse = np.mean(cross_val_score(gradient_boosting, X, y,scoring='neg_mean_squared_error', cv=15,  n_jobs=2))
print("Method name = {}".format("GradientBoostingRegressor"))
print ('Mean squared error: {}'.format(abs(mse)))
print ('Precission: {} %'. format(abs(precission)))

Method name = GradientBoostingRegressor
Mean squared error: 72191442799.6
Precission: 63.9852453811 %


# AdaBoost

In [92]:
from sklearn.ensemble import AdaBoostRegressor

#X, y = data_training3, data_training3.price_aprox_usd  ###Da menos % usando todo el dataset
X,y = data_training3.loc[(data_training3.year >= 2016)], (data_training3.loc[(data_training3.year >= 2016)]).price_aprox_usd
X = X[columnas_corr]


for estimator in [DecisionTreeRegressor(),RandomForestRegressor(),ExtraTreesRegressor()]:
    for val_loss in ['linear','square','exponential']:
        ada_boost= AdaBoostRegressor(loss=val_loss,base_estimator=estimator)
        ada_boost.fit(X,y)

        precission = ada_boost.score(X,y) * 100
        mse = np.mean(cross_val_score(ada_boost, X, y,scoring='neg_mean_squared_error', cv=15,  n_jobs=2))
        print("Method name = {}, Estimator = {}, Loss = {}".format("AdaBoostRegressor",estimator,val_loss))
        print ('Mean squared error: {}'.format(abs(mse)))
        print ('Precission: {} %'. format(abs(precission)))
        print ("")
        print ("")

Method name = AdaBoostRegressor, Estimator = DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'), Loss = linear
Mean squared error: 75355433004.0
Precission: 91.1804061773 %
Method name = AdaBoostRegressor, Estimator = DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'), Loss = square
Mean squared error: 93092409724.5
Precission: 85.5269669079 %
Method name = AdaBoostRegressor, Estimator = DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           

# El mejor resultado fue con Method name = AdaBoostRegressor, Estimator = ExtraTreesRegressor, Loss = linear Mean squared error: 80528523478.4
# Precission: 92.4744716745 %

In [94]:
X,y = data_training3.loc[(data_training3.year >= 2016)], (data_training3.loc[(data_training3.year >= 2016)]).price_aprox_usd
X = X[columnas_corr]

estimator = ExtraTreesRegressor()
val_loss = "linear"
ada_boost= AdaBoostRegressor(loss=val_loss,base_estimator=estimator)
ada_boost.fit(X,y)

precission = ada_boost.score(X,y) * 100
mse = np.mean(cross_val_score(ada_boost, X, y,scoring='neg_mean_squared_error', cv=15,  n_jobs=2))
print("Method name = {}, Estimator = {}, Loss = {}".format("AdaBoostRegressor","ExtraTreesRegressor",val_loss))
print ('Mean squared error: {}'.format(abs(mse)))
print ('Precission: {} %'. format(abs(precission)))

Method name = AdaBoostRegressor, Estimator = ExtraTreesRegressor, Loss = linear
Mean squared error: 77537279171.4
Precission: 92.7413825618 %


# Uso RandomizedSearch y GridSearch

#### Medio que es fruta... no encuentra nada bueno

In [53]:
from sklearn.ensemble import RandomForestRegressor

#X, y = data_training3, data_training3.price_aprox_usd
columnas2 = ['lat', 'lon', 'surface_total_in_m2', 'rooms', 'year', 'month',
 'property_type_code', 'place_name_code', 'state_name_code', 'cochera', 'pileta', 'vigilancia', 'gimnasio',
 'patio', 'terraza', 'parrilla', 'lavadero']
X,y = data_training3.loc[(data_training3.year >= 2016)], (data_training3.loc[(data_training3.year >= 2016)]).price_aprox_usd
X = X[columnas_corr]

# build a classifier
clf = RandomForestRegressor()


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  (results['mean_test_score'][candidate])*100,
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None,5,10,1],
              "max_features": [0.2,0.4,0.6,0.8,1.0],
              "n_estimators" : [10,25,50,80,100],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [2, 3, 10],
              "criterion": ["mse","friedman_mse"],
              "bootstrap": [True, False]}

# run randomized search
random_search = RandomizedSearchCV(estimator = clf, param_distributions=param_dist, n_jobs = 2, cv = 15)

start = time.time()
print ("Fitting... Inicio: {} ".format(time.strftime("%X")))
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates parameter settings." % ((time.time() - start), len(random_search.cv_results_['params'])))
report(random_search.cv_results_)



Fitting... Inicio: 10:12:34 
RandomizedSearchCV took 923.98 seconds for 10 candidates parameter settings.
Model with rank: 1
Mean validation score: 54.477 (std: 0.128)
Parameters: {'bootstrap': True, 'min_samples_leaf': 3, 'n_estimators': 80, 'max_features': 0.6, 'criterion': 'mse', 'min_samples_split': 3, 'max_depth': None}

Model with rank: 2
Mean validation score: 54.051 (std: 0.134)
Parameters: {'bootstrap': True, 'min_samples_leaf': 3, 'n_estimators': 80, 'max_features': 0.8, 'criterion': 'friedman_mse', 'min_samples_split': 3, 'max_depth': None}

Model with rank: 3
Mean validation score: 52.484 (std: 0.138)
Parameters: {'bootstrap': False, 'min_samples_leaf': 3, 'n_estimators': 50, 'max_features': 0.8, 'criterion': 'mse', 'min_samples_split': 2, 'max_depth': None}



In [54]:
columnas2 = ['lat', 'lon', 'surface_total_in_m2', 'rooms', 'year', 'month',
 'property_type_code', 'place_name_code', 'state_name_code', 'cochera', 'pileta', 'vigilancia', 'gimnasio',
 'patio', 'terraza', 'parrilla', 'lavadero']

X,y = data_training3.loc[(data_training3.year >= 2016)], (data_training3.loc[(data_training3.year >= 2016)]).price_aprox_usd

#X, y = data_training3, data_training3.price_aprox_usd
X = X[columnas_corr]

# build a classifier
tree = DecisionTreeRegressor()


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  (results['mean_test_score'][candidate])*100,
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None,5,10,1],
              "max_features": [0.2,0.4,0.6,0.8,1.0],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [2, 3, 10],
              "criterion": ["mse","friedman_mse"]}

# run randomized search
random_search = RandomizedSearchCV(estimator = tree, param_distributions=param_dist, n_jobs= 2,cv = 15)

start = time.time()
print ("Fitting... Inicio: {} ".format(time.strftime("%X")))
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates parameter settings." % ((time.time() - start), len(random_search.cv_results_['params'])))
report(random_search.cv_results_)



Fitting... Inicio: 10:29:16 
RandomizedSearchCV took 18.72 seconds for 10 candidates parameter settings.
Model with rank: 1
Mean validation score: 42.929 (std: 0.123)
Parameters: {'min_samples_split': 10, 'max_features': 0.8, 'criterion': 'friedman_mse', 'max_depth': 10, 'min_samples_leaf': 3}

Model with rank: 2
Mean validation score: 42.734 (std: 0.191)
Parameters: {'min_samples_split': 3, 'max_features': 0.8, 'criterion': 'friedman_mse', 'max_depth': 10, 'min_samples_leaf': 2}

Model with rank: 3
Mean validation score: 39.319 (std: 0.225)
Parameters: {'min_samples_split': 3, 'max_features': 0.6, 'criterion': 'mse', 'max_depth': 10, 'min_samples_leaf': 2}



In [61]:
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  (results['mean_test_score'][candidate])*100,
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

# use a full grid over all parameters
param_grid = {"max_depth": [3, None,5,10,1],
              "max_features": [0.2,0.4,0.6,0.8,1.0],
              "splitter" : ["random","best"],
              "min_samples_split": [2, 3],
              "min_samples_leaf": [1, 3, 10],
              "criterion": ["mse","friedman_mse"]}

columnas2 = ['lat', 'lon', 'surface_total_in_m2', 'rooms', 'year', 'month',
 'property_type_code', 'place_name_code', 'state_name_code', 'cochera', 'pileta', 'vigilancia', 'gimnasio',
 'patio', 'terraza', 'parrilla', 'lavadero']

#X,y = data_training3, data_training3.price_aprox_usd
X,y = data_training3.loc[(data_training3.year >= 2016)], (data_training3.loc[(data_training3.year >= 2016)]).price_aprox_usd
X = X[columnas_corr]

tree = DecisionTreeRegressor()

# run grid search
grid_search = GridSearchCV(estimator = tree, param_grid=param_grid,n_jobs= 2,cv = 15)
start = time.time()
print ("Fitting... Inicio: {} ".format(time.strftime("%X")))
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time.time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)


Fitting... Inicio: 10:33:05 
GridSearchCV took 1273.38 seconds for 600 candidate parameter settings.
Model with rank: 1
Mean validation score: 47.792 (std: 0.117)
Parameters: {'splitter': 'best', 'min_samples_leaf': 10, 'min_samples_split': 3, 'criterion': 'friedman_mse', 'max_features': 0.6, 'max_depth': None}

Model with rank: 2
Mean validation score: 47.567 (std: 0.118)
Parameters: {'splitter': 'best', 'min_samples_leaf': 10, 'min_samples_split': 3, 'criterion': 'mse', 'max_features': 0.8, 'max_depth': None}

Model with rank: 3
Mean validation score: 46.815 (std: 0.118)
Parameters: {'splitter': 'best', 'min_samples_leaf': 10, 'min_samples_split': 2, 'criterion': 'friedman_mse', 'max_features': 1.0, 'max_depth': None}



In [143]:
from sklearn.ensemble import ExtraTreesRegressor

estimators =  ExtraTreesRegressor()


columnas2 = ['lat', 'lon', 'surface_total_in_m2', 'rooms', 'year', 'month',
 'property_type_code', 'place_name_code', 'state_name_code', 'cochera', 'pileta', 'vigilancia', 'gimnasio',
 'patio', 'terraza', 'parrilla', 'lavadero']
X,y = data_training3.loc[(data_training3.year >= 2016)], (data_training3.loc[(data_training3.year >= 2016)]).price_aprox_usd
X = X[columnas_corr]

# Utility function to report best scores
def report(results, n_top=5):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None,5,10,1],
              "n_estimators" : [10,25,50,80,100],
              "max_features": [0.2,0.4,0.6,0.8,1.0],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [2, 3, 10],
              "criterion": ["mse","friedman_mse"]}

# run randomized search
random_search = RandomizedSearchCV(estimator = estimators, param_distributions=param_dist, n_jobs= 2,cv = 15)

start = time.time()
print ("Fitting... Inicio: {} ".format(time.strftime("%X")))
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates parameter settings." % ((time.time() - start), len(random_search.cv_results_['params'])))
report(random_search.cv_results_)


Fitting... Inicio: 16:54:37 
RandomizedSearchCV took 264.79 seconds for 10 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.448 (std: 0.102)
Parameters: {'min_samples_leaf': 10, 'n_estimators': 80, 'max_features': 0.8, 'criterion': 'mse', 'min_samples_split': 3, 'max_depth': None}

Model with rank: 2
Mean validation score: 0.397 (std: 0.092)
Parameters: {'min_samples_leaf': 10, 'n_estimators': 80, 'max_features': 0.6, 'criterion': 'friedman_mse', 'min_samples_split': 2, 'max_depth': None}

Model with rank: 3
Mean validation score: 0.172 (std: 0.044)
Parameters: {'min_samples_leaf': 3, 'n_estimators': 10, 'max_features': 0.8, 'criterion': 'mse', 'min_samples_split': 10, 'max_depth': 5}



# Hacemos las predicciones del set de test

In [64]:
testingSet.dropna(axis = 0, subset = ['lat', 'lon', 'surface_total_in_m2', 'rooms', 'year', 'month',
 'property_type_code', 'place_name_code', 'state_name_code', 'cochera', 'pileta', 'vigilancia', 'gimnasio',
 'patio', 'terraza', 'parrilla', 'lavadero'],how ='any',inplace=True)

In [53]:
testingSet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9387 entries, 0 to 14165
Data columns (total 29 columns):
id                         9387 non-null int64
property_type              9387 non-null object
operation                  9387 non-null object
place_name                 9387 non-null object
place_with_parent_names    9387 non-null object
country_name               9387 non-null object
state_name                 9387 non-null object
lat-lon                    9387 non-null object
lat                        9387 non-null float64
lon                        9387 non-null float64
surface_total_in_m2        9387 non-null float64
surface_covered_in_m2      9094 non-null float64
floor                      1271 non-null float64
rooms                      9387 non-null float64
expenses                   1620 non-null object
description                9387 non-null object
cochera                    9387 non-null int64
pileta                     9387 non-null int64
vigilancia               

In [65]:
testingSet.drop(['expenses','floor','surface_covered_in_m2'],axis = 1, inplace=True)

In [77]:
testingSet.loc[:,'price_aprox_usd'] = testingSet[columnas_corr].apply(lambda x: regression_tree.predict([x])[0],axis = 1)

In [78]:
resultado_tree_regressor = testingSet.loc[:,['id','price_aprox_usd']]

In [79]:
resultado_tree_regressor.to_csv('Predicciones/Tree_Regressor2.csv', index = False)

In [80]:
resultado_tree_regressor

,id,price_aprox_usd
0,3632,5.225000e+05
2,2263404,1.928496e+05
8,2263410,1.355000e+05
13,2263416,6.629000e+05
29,2263432,1.119370e+05
30,2263433,1.119370e+05
31,2263434,1.119370e+05
32,2263436,1.119370e+05
33,2263437,1.119370e+05
35,2263439,1.119370e+05


In [73]:
testingSet.loc[:,'price_aprox_usd'] = testingSet[columnas2].apply(lambda x: extra_tree.predict([x])[0],axis = 1)

In [74]:
resultado_extra_tree = testingSet.loc[:,['id','price_aprox_usd']]

In [76]:
resultado_extra_tree.to_csv('Predicciones/Extra_Tree2.csv', index = False)

In [75]:
resultado_extra_tree

,id,price_aprox_usd
0,3632,570000.000000
2,2263404,206200.000000
8,2263410,141800.000000
13,2263416,559150.000000
29,2263432,102151.500000
30,2263433,102151.500000
31,2263434,102151.500000
32,2263436,102151.500000
33,2263437,102151.500000
35,2263439,102151.500000


In [67]:
testingSet.loc[:,'price_aprox_usd'] = testingSet[columnas2].apply(lambda x: random_forest.predict([x])[0],axis = 1)

In [68]:
resultado_random_forest = testingSet.loc[:,['id','price_aprox_usd']]

In [71]:
resultado_random_forest.to_csv('Predicciones/Random_Forest.csv', index = False)

In [69]:
resultado_random_forest

,id,price_aprox_usd
0,3632,578000.000000
2,2263404,213660.000000
8,2263410,132700.000000
13,2263416,883000.000000
29,2263432,83898.990304
30,2263433,83898.990304
31,2263434,83898.990304
32,2263436,83898.990304
33,2263437,83898.990304
35,2263439,83898.990304


In [95]:
testingSet.loc[:,'price_aprox_usd'] = testingSet[columnas_corr].apply(lambda x: ada_boost.predict([x])[0],axis = 1)

In [96]:
resultado_ada_boost = testingSet.loc[:,['id','price_aprox_usd']]

In [98]:
resultado_ada_boost.to_csv('Predicciones/Ada_Boost.csv', index = False)

In [97]:
resultado_ada_boost

,id,price_aprox_usd
0,3632,510000.000000
2,2263404,195783.640000
8,2263410,136200.000000
13,2263416,491000.000000
29,2263432,111937.000000
30,2263433,111937.000000
31,2263434,111937.000000
32,2263436,111937.000000
33,2263437,111937.000000
35,2263439,111937.000000
